In [2]:
import glob
import numpy as np
import os

from obspy import read
from obspy import read_events

import matplotlib.pyplot as plt

# import time

In [15]:
## Frequencies for filter
fmin = 1/20. #Hz
fmax = 1/10.  #Hzseis

dist_range_0 = 90
dist_range_1 = 100
dist_range_2 = 110
dist_range_3 = 120
dist_range_4 = 130
dist_range_5 = 140

azi_min = 0
azi_max = 360

time_min = -40
time_max = 100

linewidth = 0.2
norm_constant = 0.4

if_round = True

switch_yaxis = True

def PLOTSeismogram(EVENTNAME,atAziBound):
    RePlot = False
    print(EVENTNAME)

    DataStream = read(f"../Data/ProcessedSecondRequest/{EVENTNAME}.PICKLE",format='PICKLE')
    DataStream.filter('bandpass', freqmin=fmin,freqmax=fmax)
    DataStream.differentiate()

    CatFind = glob.glob(f"../Data/CMTSOLUTION/{EVENTNAME[0:12]}*.CMTSOLUTION")
    cat = read_events(CatFind[0])

    EventDepth = cat[0].origins[0].depth/1000.
    EventLat, EventLat = cat[0].origins[0].latitude, cat[0].origins[0].longitude
    EventTime = cat[0].origins[0].time

    catinfo = "%s" %cat
    EventInfo = "%s %.1f km" %(catinfo.split("\n")[-1], EventDepth, )
    print(EventInfo)
    
    # try:
    #     SyntheticStream = read("./Data/Synthetics/Synthetic%s.mseed" %EVENTNAME,format='mseed')
    #     SyntheticStream.filter('bandpass', freqmin=fmin,freqmax=fmax)
    #     SyntheticStream.differentiate()
    #     SyntheticLoad = True
    # except:
    #     print("Load synthetic data failed...")
    #     SyntheticLoad = False

    # Plot data
    fig, axs = plt.subplots(1,5,sharey=True)
    fig.set_size_inches(15,8)

    # plt.xticks(np.arange(time_min, time_max, step=20))
    plt.yticks(np.arange(0,361,10))

    for ax in axs:
        ax.set_xticks(np.arange(time_min, time_max, step=20))
        ax.set_xlim(time_min,time_max)
        ax.set_xlabel('Time [s]')
        ax.tick_params(axis='both', which='major', labelsize=10, length=5, width=1)

        for basetime in np.arange(time_min, time_max, step=20): ax.axvline(x=basetime,color='k',linewidth=linewidth)

    norm = np.max(DataStream.select(component="T")[0].data) / norm_constant

    azis=[]
    dists=[]
    ratio0_list=[]
    ratio1_list=[]
    SNR_list=[]
    for itrace, trace in enumerate(DataStream.select(component="T")):
        TraceName = trace.stats.network + '.' + trace.stats.station
        dist = trace.stats.distance_in_degrees
        azi = trace.stats.azimuth

        if atAziBound == True:
            if azi < 100:
                azi = azi + 360
        # print("dist %.1f azi %.1f" %(dist, azi))
        # Split seismograms by distance range (this needs to be adapted per Event to produce a reasonable plot.
        if dist < dist_range_1 and dist > dist_range_0:
            i_axis = 0
        elif dist < dist_range_2 and dist > dist_range_1:
            i_axis = 1
        elif dist < dist_range_3 and dist > dist_range_2:
            i_axis = 2
        elif dist < dist_range_4 and dist > dist_range_3:
            i_axis = 3
        elif dist < dist_range_5 and dist > dist_range_4:
            i_axis = 4
        else:
            continue
        try:
            phase_time = trace.stats.traveltimes['S'] or trace.stats.traveltimes['Sdiff']
            align_time = phase_time
            # print("align time: ", align_time)
        except:
            print('ERROR loading travetime')
            print(trace)
            continue

        
        if if_round:
            azi = np.round(azi)    
        
        # Plot real data
        plot_time = trace.times(reftime=EventTime)-align_time

        try:
            NormValue = np.max(trace.data[np.logical_and(plot_time>time_min, plot_time<time_max)]) / norm_constant
        except:
            continue
        # NoiseValue = np.max(trace.data[np.logical_and(plot_time>-40, plot_time<-20)]) / norm_constant
        plot_data = trace.data / NormValue + azi
        axs[i_axis].plot(plot_time, plot_data,'k',linewidth=linewidth)

        azis.append(azi) # list all azimuths
        dists.append(dist)

    azis = np.array(azis)
    dists = np.array(dists)



    # axs[0,0].set_xlim(time_min,time_max)
    axs[0].set_ylim(azis.min()-3,azis.max()+3)

    print("Plot azimuth range: ", azis.max()-azis.min()+6)
    if azis.max()-azis.min() > 300:
        RePlot = True

    axs[0].set_ylabel('Azimuth [$^{\circ}$]')
    axs[0].set_title(' Sdiff dist %d - %d' %(dist_range_0,dist_range_1))
    axs[1].set_title(' Sdiff dist %d - %d' %(dist_range_1,dist_range_2))
    axs[2].set_title(' Sdiff dist %d - %d' %(dist_range_2,dist_range_3))
    axs[3].set_title(' Sdiff dist %d - %d' %(dist_range_3,dist_range_4))
    axs[4].set_title(' Sdiff dist %d - %d' %(dist_range_4,dist_range_5))
    axs[0].set_xlabel('Time [s]')
    axs[1].set_xlabel('Time [s]')
    axs[2].set_xlabel('Time [s]')
    axs[3].set_xlabel('Time [s]')
    axs[4].set_xlabel('Time [s]')

    if switch_yaxis:
        axs[0].invert_yaxis()
    fig.suptitle(EventInfo,fontsize = 20)
    
    plt.savefig(f"../Plot/{EVENTNAME}seismogram.png",format='png')
    plt.close('all')

    return RePlot

In [16]:
for EVENTPATH in glob.glob("../Data/ProcessedSecondRequest/*.PICKLE")[0:1]:
    EVENTNAME = EVENTPATH.split('/')[-1].split('.')[0]
    # try:
    RePlot = PLOTSeismogram(EVENTNAME, atAziBound=False)
    print('PLOTSeismogram %s FINISHED!' %EVENTNAME)
    # except:
    #     print('PLOTSeismogram %s ERROR' %EVENTNAME)

    if RePlot:
        print("Azimuth at 360 degree boundary, now RePlot!")
        PLOTSeismogram(EVENTNAME, atAziBound=True)

20210822004513
2021-08-22T00:45:13.800000Z | -60.330,  -23.600 | 6.69 mw 12.0 km
Plot azimuth range:  44.0
PLOTSeismogram 20210822004513 FINISHED!
